In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_zarr',walltime='00:20:00',job_extra=['--constraint=BDW28','--exclusive','--nodes=1'],memory='20GB') 
print(cluster.job_script()) 
  

#!/usr/bin/env bash

#SBATCH -J make_zarr
#SBATCH -n 1
#SBATCH --cpus-per-task=28
#SBATCH --mem=19G
#SBATCH -t 00:20:00
#SBATCH --constraint=BDW28
#SBATCH --exclusive
#SBATCH --nodes=1
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://195.83.183.132:46569 --nthreads 28 --memory-limit 20.00GB --name make_zarr--${JOB_ID}-- --death-timeout 60



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
cluster.scale(1) 
#cluster.adapt(minimum=1, maximum=4) 


In [3]:
import xarray as xr 
import dask 
import dask.threaded 
import dask.multiprocessing 
from dask.distributed import Client 
import zarr 
import numpy as np 
import os 
import time 

import glob

In [1]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'

tfiles = sorted(glob.glob(data_dir + '*/eNATL60-BLBT02_1h_*_gridT_2009070?-2009070?.nc'))
sfiles = sorted(glob.glob(data_dir + '*/eNATL60-BLBT02_1h_*_gridS_2009070?-2009070?.nc'))

def non_time_coords(ds):
    return [v for v in ds.data_vars
            if 'time_counter' not in ds[v].dims]

def drop_non_essential_vars_pop(ds):
    return ds.drop(non_time_coords(ds))

%time dst=xr.open_mfdataset(tfiles,parallel=True,concat_dim='time_counter',decode_cf=False,preprocess=drop_non_essential_vars_pop,chunks={'deptht':1,'time_counter':1}) 
%time dss=xr.open_mfdataset(sfiles,parallel=True,concat_dim='time_counter',decode_cf=False,preprocess=drop_non_essential_vars_pop,chunks={'deptht':1,'time_counter':1})


NameError: name 'glob' is not defined